In [1]:
import optuna
import os
from pathlib import Path
import kaggle
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

SEED = 42
SPLITS = 5
VERSION = 3
OUTPUT = f'hist_grad_boost_clf_submisson_v{VERSION}.csv'
N_TRIALS = 30

np.random.seed(SEED)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

In [5]:
partial_pipeline = make_pipeline(ColumnTransformer([('drop', 'drop',
                                  ['iv(g)', 't', 'b', 'n',
                                   'lOCode', 'v', 'branchCount',
                                   'e', 'i', 'lOComment'])],
                                remainder='passthrough'), SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), PowerTransformer())

In [6]:
def objective(trial):
    params = {
        'random_state': trial.suggest_categorical('random_state', [SEED]),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'l2_regularization': trial.suggest_int('l2_regularization', 0, 100),
        'max_iter': trial.suggest_int('max_iter', 10, 10_000),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 500),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 300),
        'max_bins': trial.suggest_int('max_bins', 32, 255),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced'])
    }
    pipeline = make_pipeline(partial_pipeline, HistGradientBoostingClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=SPLITS, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

[I 2023-10-16 14:08:38,850] A new study created in memory with name: no-name-ac063050-4fee-4d61-8104-c94df3e0a139
Best trial: 0. Best value: 0.783083:   3%|█████▍                                                                                                                                                              | 1/30 [00:14<07:13, 14.96s/it]

[I 2023-10-16 14:08:53,811] Trial 0 finished with value: 0.7830826929961583 and parameters: {'random_state': 42, 'learning_rate': 0.016147244964703066, 'l2_regularization': 31, 'max_iter': 579, 'max_leaf_nodes': 265, 'min_samples_leaf': 33, 'max_bins': 113, 'class_weight': None}. Best is trial 0 with value: 0.7830826929961583.


Best trial: 0. Best value: 0.783083:   7%|██████████▉                                                                                                                                                         | 2/30 [00:20<04:27,  9.57s/it]

[I 2023-10-16 14:08:59,600] Trial 1 finished with value: 0.7822329003357007 and parameters: {'random_state': 42, 'learning_rate': 0.04281011805863016, 'l2_regularization': 4, 'max_iter': 6086, 'max_leaf_nodes': 480, 'min_samples_leaf': 258, 'max_bins': 41, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7830826929961583.


Best trial: 2. Best value: 0.783769:  10%|████████████████▍                                                                                                                                                   | 3/30 [00:25<03:14,  7.19s/it]

...

[I 2023-10-16 14:12:01,517] Trial 27 finished with value: 0.7837531806688487 and parameters: {'random_state': 42, 'learning_rate': 0.03869978488539974, 'l2_regularization': 54, 'max_iter': 6393, 'max_leaf_nodes': 215, 'min_samples_leaf': 47, 'max_bins': 104, 'class_weight': None}. Best is trial 12 with value: 0.7861688418968245.


Best trial: 12. Best value: 0.786169:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 29/30 [03:27<00:05,  5.45s/it]

[I 2023-10-16 14:12:06,035] Trial 28 finished with value: 0.7850467591210757 and parameters: {'random_state': 42, 'learning_rate': 0.049921907572925125, 'l2_regularization': 19, 'max_iter': 8406, 'max_leaf_nodes': 40, 'min_samples_leaf': 117, 'max_bins': 48, 'class_weight': None}. Best is trial 12 with value: 0.7861688418968245.


Best trial: 12. Best value: 0.786169: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [03:31<00:00,  7.06s/it]

[I 2023-10-16 14:12:10,569] Trial 29 finished with value: 0.783053790414532 and parameters: {'random_state': 42, 'learning_rate': 0.08287264443906442, 'l2_regularization': 33, 'max_iter': 9237, 'max_leaf_nodes': 197, 'min_samples_leaf': 22, 'max_bins': 116, 'class_weight': None}. Best is trial 12 with value: 0.7861688418968245.


In [7]:
study.best_params, study.best_value

({'random_state': 42,
  'learning_rate': 0.04914413927409843,
  'l2_regularization': 33,
  'max_iter': 9946,
  'max_leaf_nodes': 9,
  'min_samples_leaf': 13,
  'max_bins': 80,
  'class_weight': None},
 0.7861688418968245)

In [8]:
pipeline = make_pipeline(partial_pipeline, HistGradientBoostingClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('drop',
                                                                   'drop',
                                                                   ['iv(g)',
                                                                    't', 'b',
                                                                    'n',
                                                                    'lOCode',
                                                                    'v',
                                                                    'branchCount',
                                                                    'e', 'i',
                                                                    'lOComment'])])),
                                 ('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('powertransformer', PowerTransformer())])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(l2_regularization=33,
                                                learning_rate=0.04914413927409843,
                                                max_bins=80, max_iter=9946,
                                                max_leaf_nodes=9,
                                                min_samples_leaf=13,
                                                random_state=42))])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)